In [38]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST

In [39]:
# 开启gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device) # cuda

cuda


In [40]:
# 载入数据集
dataset = FashionMNIST(
    root='data',
    train=True,
    transform=transforms.ToTensor(),
    download=True)

data_loader = DataLoader(
    dataset=dataset,
    batch_size=64,
    shuffle=True)

In [41]:
# 定义变分自编码器
class VAE(nn.Module):
    def __init__(self, image_size=784):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(784, 400)
        self.fc2 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(400, 20)
        self.fc4 = nn.Linear(20, 400)
        self.fc5 = nn.Linear(400, 784)

    def encode(self, x):
        h = F.relu(self.fc1(x))
        return self.fc2(h), self.fc3(h)

    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var / 2)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = F.relu(self.fc4(z))
        return F.sigmoid(self.fc5(h))

    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_reconst = self.decode(z)
        return x_reconst, mu, log_var


vaeModel = VAE().to(device)
optimizer = torch.optim.Adam(vaeModel.parameters(), lr=1e-4)


In [42]:
for epoch in range(20):
    for i, (x, _) in enumerate(data_loader):

        x = x.to(device).view(-1, 784)
        x_reconst, mu, log_var = vaeModel(x)
        
        reconst_loss = F.binary_cross_entropy(x_reconst, x, size_average=False)
        kl_div = - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        
        loss = reconst_loss + kl_div
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print("Epoch[{}/{}],Reconst Loss: {:.4f}".format(epoch+1, 20,reconst_loss.item()))
    
    with torch.no_grad():
        out, _, _ = vaeModel(x)
        x_concat = torch.cat([x.view(-1, 1, 28, 28), out.view(-1, 1, 28, 28)], dim=3)
        save_image(x_concat, os.path.join('imagesAfterVAE', 'new-{}.png'.format(epoch+1)))


Epoch[1/20],Reconst Loss: 24479.9473
Epoch[1/20],Reconst Loss: 21513.4297
Epoch[1/20],Reconst Loss: 20585.4922
Epoch[1/20],Reconst Loss: 18553.8828
Epoch[1/20],Reconst Loss: 17870.0391
Epoch[1/20],Reconst Loss: 19336.3418
Epoch[1/20],Reconst Loss: 17340.0254
Epoch[1/20],Reconst Loss: 17771.6758
Epoch[1/20],Reconst Loss: 16845.7344
Epoch[2/20],Reconst Loss: 16551.6328
Epoch[2/20],Reconst Loss: 16829.4922
Epoch[2/20],Reconst Loss: 16932.4824
Epoch[2/20],Reconst Loss: 16275.6807
Epoch[2/20],Reconst Loss: 15964.1123
Epoch[2/20],Reconst Loss: 16183.8105
Epoch[2/20],Reconst Loss: 15656.6318
Epoch[2/20],Reconst Loss: 15983.4941
Epoch[2/20],Reconst Loss: 15815.7471
Epoch[3/20],Reconst Loss: 16181.8027
Epoch[3/20],Reconst Loss: 16147.0557
Epoch[3/20],Reconst Loss: 15249.6562
Epoch[3/20],Reconst Loss: 16408.1953
Epoch[3/20],Reconst Loss: 15418.3320
Epoch[3/20],Reconst Loss: 15978.2480
Epoch[3/20],Reconst Loss: 16167.6992
Epoch[3/20],Reconst Loss: 16204.2617
Epoch[3/20],Reconst Loss: 14011.7354
E